# Try to remove last layer with Keras model BirdNET V2.1

In [1]:
import tensorflow as tf
from tensorflow import keras
model = tf.keras.models.load_model("/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model/")

2023-10-07 15:08:37.064999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 15:08:37.112944: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 15:08:37.114072: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 15:08:37.954890: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Layers before removal

In [2]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 INPUT (InputLayer)          [(None, 144000)]             0         []                            
                                                                                                  
 ADVANCED_SPEC1 (LinearSpec  (None, 128, 513, 1)          1         ['INPUT[0][0]']               
 Layer)                                                                                           
                                                                                                  
 BNORM_SPEC_NOQUANT (BatchN  (None, 128, 513, 1)          4         ['ADVANCED_SPEC1[0][0]']      
 ormalization)                                                                                    
                                                                                              

# Remove last fully connected Layer  "CLASS_DENSE_1 (Dense)"  with shape     (None, 2434)

In [3]:
from keras.models import Sequential
from tensorflow.keras.models import Model

model2 = Model(model.input, model.layers[-2].output)

In [4]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 INPUT (InputLayer)          [(None, 144000)]             0         []                            
                                                                                                  
 ADVANCED_SPEC1 (LinearSpec  (None, 128, 513, 1)          1         ['INPUT[0][0]']               
 Layer)                                                                                           
                                                                                                  
 BNORM_SPEC_NOQUANT (BatchN  (None, 128, 513, 1)          4         ['ADVANCED_SPEC1[0][0]']      
 ormalization)                                                                                    
                                                                                              

In [5]:
# Convert the keras model to a tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
tflite_model2 = converter.convert()

print("Finished Converting")


INFO:tensorflow:Assets written to: /tmp/tmpd4s2nu6_/assets


INFO:tensorflow:Assets written to: /tmp/tmpd4s2nu6_/assets
2023-10-07 15:11:16.351078: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-10-07 15:11:16.351180: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-10-07 15:11:16.352051: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpd4s2nu6_
2023-10-07 15:11:16.384959: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-10-07 15:11:16.385045: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpd4s2nu6_
2023-10-07 15:11:16.450268: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-10-07 15:11:16.476855: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-10-07 15:11:16.914069: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

Finished Converting


In [6]:
#save
with open('/home/jovyan/20230717_test_layer_remove.tflite', 'wb') as f:
    f.write(tflite_model2)
print("Finished Saving")

Finished Saving


# Analyze the saved tflite models, original vs. removed dense layer

In [7]:
path_to_original_V_2_1 = "/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model_FP32.tflite"
tf.lite.experimental.Analyzer.analyze(model_path=path_to_original_V_2_1,
                                      model_content=None,
                                      gpu_compatibility=False)



=== /home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model_FP32.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the REDUCE_MIN op takes
tensor #0 and tensor #117 as input and produces tensor #143 as output.

Subgraph#0 main(T#0) -> [T#404]
  Op#0 REDUCE_MIN(T#0, T#117[1]) -> [T#143]
  Op#1 SUB(T#0, T#143) -> [T#144]
  Op#2 REDUCE_MAX(T#144, T#117[1]) -> [T#145]
  Op#3 DIV(T#144, T#145) -> [T#146]
  Op#4 SHAPE(T#146) -> [T#147]
  Op#5 SPLIT_V(T#147, T#116[1, 1, 0], T#120[0]) -> [T#148, T#149, T#150]
  Op#6 RESHAPE(T#149, T#121[]) -> [T#151]
  Op#7 FLOOR_DIV(T#151, T#122[8]) -> [T#152]
  Op#8 PACK(T#152, T#122[8]) -> [T#153]
  Op#9 MUL(T#152, T#122[8]) -> [T#154]
  Op#10 RESHAPE(T#154, T#124[1]) -> [T#155]
  Op#11 CONCATENATION(T#148, T#155) -> [T#156]
  Op#12 CONCATENATION(T#148, T#153) -> [T#157]
  Op#13 STRIDED_SLICE(T#146, T#125[0, 0], T#156, T#118[1,

In [8]:
path_to_testexport = "/home/jovyan/20230717_test_layer_remove.tflite"
tf.lite.experimental.Analyzer.analyze(model_path=path_to_testexport,
                                      model_content=None,
                                      gpu_compatibility=False)

=== /home/jovyan/20230717_test_layer_remove.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the REDUCE_MIN op takes
tensor #0 and tensor #116 as input and produces tensor #142 as output.

Subgraph#0 main(T#0) -> [T#402]
  Op#0 REDUCE_MIN(T#0, T#116[1]) -> [T#142]
  Op#1 SUB(T#0, T#142) -> [T#143]
  Op#2 REDUCE_MAX(T#143, T#116[1]) -> [T#144]
  Op#3 DIV(T#143, T#144) -> [T#145]
  Op#4 SHAPE(T#145) -> [T#146]
  Op#5 SPLIT_V(T#146, T#124[1, 1, 0], T#136[0]) -> [T#147, T#148, T#149]
  Op#6 RESHAPE(T#148, T#135[]) -> [T#150]
  Op#7 FLOOR_DIV(T#150, T#134[8]) -> [T#151]
  Op#8 PACK(T#151, T#134[8]) -> [T#152]
  Op#9 MUL(T#151, T#134[8]) -> [T#153]
  Op#10 RESHAPE(T#153, T#119[1]) -> [T#154]
  Op#11 CONCATENATION(T#147, T#154) -> [T#155]
  Op#12 CONCATENATION(T#147, T#152) -> [T#156]
  Op#13 STRIDED_SLICE(T#145, T#133[0, 0], T#155, T#125[1, 1]) -> [T#157]
  Op#14 RESHAPE(T#157, T#156) -> [T#158

---
## -> also removed Tensors from Dense Layer (T#142 and T#135)